In [33]:
import os
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split,  cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier # keras wrapper for sklearn

In [2]:
# Reading the data (sample)
data_s = pd.read_csv('train_sample.csv')#.fillna('')

In [3]:
data_s.sort_values(by='attributed_time', ascending = False)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
28970,346097,19,16,0,213,2017-11-09 15:27:07,2017-11-09 15:28:15,1
5111,303875,71,1,19,3,2017-11-09 15:13:48,2017-11-09 15:17:13,1
93842,346361,115,1,22,203,2017-11-09 15:14:10,2017-11-09 15:15:34,1
39247,305220,19,581,24,213,2017-11-09 15:03:23,2017-11-09 15:05:43,1
27035,75443,9,1,19,466,2017-11-09 14:12:55,2017-11-09 14:53:22,1
56395,2948,45,1,2,419,2017-11-09 14:30:44,2017-11-09 14:30:54,1
71850,304428,108,1,19,243,2017-11-09 14:15:31,2017-11-09 14:22:16,1
91997,351158,29,1,19,213,2017-11-09 13:27:29,2017-11-09 13:32:51,1
37850,346860,145,50,0,320,2017-11-09 13:03:36,2017-11-09 13:04:59,1
28999,347722,19,0,0,213,2017-11-09 13:02:11,2017-11-09 13:03:34,1


In [4]:
# Checking for NANs
for col in data_s.columns:
    print('{} contains {:,} NANs'.format(col,len(data_s[data_s[col].isnull()])))

ip contains 0 NANs
app contains 0 NANs
device contains 0 NANs
os contains 0 NANs
channel contains 0 NANs
click_time contains 0 NANs
attributed_time contains 99,773 NANs
is_attributed contains 0 NANs


Only attributed time has NANs, but I am not using this column

In [5]:
# Removing attributed_time
data_s = data_s.drop('attributed_time', axis = 1)

In [6]:
# Convert click_time to datetime format
data_s.click_time = pd.to_datetime(data_s.click_time, errors = 'ignore')

In [7]:
# Determine and coding time of the day the site was accessed
# 1 = Night, 2 = Morning, 3 = Afternoon, 4 = Evening
data_s = data_s.assign(session=pd.cut(data_s.click_time.dt.hour,[-1,6,12,18,24],labels=[1, 2, 3,4 ]))
data_s = data_s.drop('click_time', 1)

In [8]:
data_s = data_s[['ip', 'app', 'device', 'os', 'channel', 'session','is_attributed']]
data_s = data_s.reset_index(drop = False)
data_s = data_s.rename(columns={'index':'click_id'})

In [9]:
data_s.head()

,click_id,ip,app,device,os,channel,session,is_attributed
0,0,87540,12,1,13,497,2,0
1,1,105560,25,1,17,259,3,0
2,2,101424,12,1,19,212,3,0
3,3,94584,13,1,13,477,1,0
4,4,68413,12,1,1,178,2,0


# Aritificial Neural Network 

### Split data into target and features

In [11]:
target = data_s.is_attributed.values
features = data_s.drop('is_attributed', 1).values
print ('Data original %d, target: %d, features: %d' % (data_s.shape[0], target.shape[0], features.shape[0]))

Data original 100000, target: 100000, features: 100000


### Split features into training, validation, and testing sets

In [12]:
X_train, X_test, y_train, y_test  = train_test_split(features, target, test_size= 0.25, random_state=1)
X_train, X_val, y_train, y_val    = train_test_split(X_train, y_train, test_size= 0.20, random_state=1)

X_train_names = X_train[:,0]
X_train = X_train[:,1:7]

X_test_names = X_test[:,0]
X_test = X_test[:,1:7]
                  
X_val_names = X_val[:,0]
X_val = X_val[:,1:7]

### Standarizing the features

In [13]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val   = sc.transform(X_val)
X_test  = sc.transform(X_test)

/Users/jpinzon/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### Building the initial model 

In [14]:
def classifier_model(activator, optimizer, initializer):
    classifier = Sequential()
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator, input_dim = 6))
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator))
    classifier.add(Dense(units = 1,  kernel_initializer = initializer, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

### Fitting the model with the training set 

In [15]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
classifier = classifier_model(activator, optimizer, initializer)
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
60000/60000 [==============================] - 25s - loss: 0.0290 - acc: 0.9978    
Epoch 2/10
60000/60000 [==============================] - 24s - loss: 0.0130 - acc: 0.9978    
Epoch 3/10
60000/60000 [==============================] - 23s - loss: 0.0128 - acc: 0.9978    
Epoch 4/10
60000/60000 [==============================] - 23s - loss: 0.0126 - acc: 0.9978    
Epoch 5/10
60000/60000 [==============================] - 24s - loss: 0.0125 - acc: 0.9978    
Epoch 6/10
60000/60000 [==============================] - 24s - loss: 0.0124 - acc: 0.9978    
Epoch 7/10
60000/60000 [==============================] - 24s - loss: 0.0124 - acc: 0.9978    
Epoch 8/10
60000/60000 [==============================] - 23s - loss: 0.0123 - acc: 0.9978    
Epoch 9/10
60000/60000 [==============================] - 23s - loss: 0.0125 - acc: 0.9978    
Epoch 10/10
60000/60000 [==============================] - 23s - loss: 0.0123 - acc: 0.9978    


### Crossvalidation on training set

In [16]:
# Using relu - as raw model above - as activator
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
cv = 10 
classifier_cv = KerasClassifier(build_fn = classifier_model, 
                                activator = activator, 
                                optimizer = optimizer,
                                initializer = initializer,
                                batch_size = 10, 
                                epochs = 10, 
                                verbose = 0)
accuracies = cross_val_score(estimator = classifier_cv, X = X_train, y = y_train, cv = cv)

mean = accuracies.mean()
variance = accuracies.std()

In [17]:
# Mean cv accuracy 
print('Mean cv accuracy (relu) = {:.4f}% +/- {:.4f}'.format(mean *100, variance *100) )

Mean cv accuracy (relu) = 99.7800% +/- 0.0452


### Validation 

In [18]:
loss, accuracy = classifier.evaluate(X_val, y_val, batch_size = 10, verbose = 0)
print("Validation set accuracy = {:.4f}%, Loss = {:.4f}".format(accuracy* 100, loss))

Validation set accuracy = 99.7533%, Loss = 0.0130


In [19]:
loss, accuracy = classifier.evaluate(X_train, y_train , batch_size = 128, verbose = 0)
print(round(accuracy*100,2))

99.78


### Optimization

Using GridSearch

In [20]:
classifier_op = KerasClassifier(build_fn = classifier_model)

parameters = {'batch_size'  : [10, 30],
              'epochs'      : [10, 20],
              'optimizer'   : ['adam', 'rmsprop'],
              'activator'   : ['relu', 'sigmoid'],
              'initializer' : ['uniform']}

grid_search = GridSearchCV(estimator = classifier_op,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/10
54000/54000 [==============================] - 22s - loss: 0.0316 - acc: 0.9977    
Epoch 2/10
54000/54000 [==============================] - 21s - loss: 0.0126 - acc: 0.9978    
Epoch 3/10
54000/54000 [==============================] - 21s - loss: 0.0125 - acc: 0.9978    
Epoch 4/10
54000/54000 [==============================] - 21s - loss: 0.0123 - acc: 0.9978    
Epoch 5/10
54000/54000 [==============================] - 21s - loss: 0.0123 - acc: 0.9978    
Epoch 6/10
54000/54000 [==============================] - 21s - loss: 0.0122 - acc: 0.9978    
Epoch 7/10
54000/54000 [==============================] - 21s - loss: 0.0122 - acc: 0.9978    
Epoch 8/10
54000/54000 [==============================] - 21s - loss: 0.0122 - acc: 0.9978    
Epoch 9/10
54000/54000 [==============================] - 21s - loss: 0.0121 - acc: 0.9978    
Epoch 10/10
54000/54000 [==============================] - 22s - loss: 0.0306 - acc: 0.9979    
Epoch 2/10
54000/54000 [=========================

54000/54000 [==============================] - 22s - loss: 0.0131 - acc: 0.9978    
Epoch 3/10
54000/54000 [==============================] - 22s - loss: 0.0129 - acc: 0.9978    
Epoch 4/10
54000/54000 [==============================] - 22s - loss: 0.0127 - acc: 0.9978    
Epoch 5/10
54000/54000 [==============================] - 22s - loss: 0.0127 - acc: 0.9978    
Epoch 6/10
54000/54000 [==============================] - 22s - loss: 0.0126 - acc: 0.9978    
Epoch 7/10
54000/54000 [==============================] - 22s - loss: 0.0126 - acc: 0.9978    
Epoch 8/10
54000/54000 [==============================] - 22s - loss: 0.0127 - acc: 0.9978    
Epoch 9/10
54000/54000 [==============================] - 22s - loss: 0.0125 - acc: 0.9978    
Epoch 10/10
54000/54000 [==============================] - 23s - loss: 0.0310 - acc: 0.9977    
Epoch 2/10
54000/54000 [==============================] - 25s - loss: 0.0128 - acc: 0.9979    
Epoch 3/10
54000/54000 [==============================] - 25

54000/54000 [==============================] - 17s - loss: 0.0212 - acc: 0.9978    
Epoch 4/10
54000/54000 [==============================] - 17s - loss: 0.0219 - acc: 0.9978    
Epoch 5/10
54000/54000 [==============================] - 17s - loss: 0.0215 - acc: 0.9978    
Epoch 6/10
54000/54000 [==============================] - 17s - loss: 0.0210 - acc: 0.9978    
Epoch 7/10
54000/54000 [==============================] - 21s - loss: 0.0190 - acc: 0.9978    
Epoch 8/10
54000/54000 [==============================] - 19s - loss: 0.0343 - acc: 0.9978    
Epoch 9/10
54000/54000 [==============================] - 20s - loss: 0.0344 - acc: 0.9978    
Epoch 10/10
54000/54000 [==============================] - 20s - loss: 0.0458 - acc: 0.9976    
Epoch 2/10
54000/54000 [==============================] - 19s - loss: 0.0212 - acc: 0.9977    
Epoch 3/10
54000/54000 [==============================] - 19s - loss: 0.0230 - acc: 0.9977    
Epoch 4/10
54000/54000 [==============================] - 19

54000/54000 [==============================] - 22s - loss: 0.0124 - acc: 0.9978    
Epoch 5/20
54000/54000 [==============================] - 22s - loss: 0.0124 - acc: 0.9978    
Epoch 6/20
54000/54000 [==============================] - 22s - loss: 0.0124 - acc: 0.9978    
Epoch 7/20
54000/54000 [==============================] - 22s - loss: 0.0123 - acc: 0.9978    
Epoch 8/20
54000/54000 [==============================] - 22s - loss: 0.0122 - acc: 0.9978    
Epoch 9/20
54000/54000 [==============================] - 22s - loss: 0.0124 - acc: 0.9978    
Epoch 10/20
54000/54000 [==============================] - 22s - loss: 0.0122 - acc: 0.9978    
Epoch 11/20
54000/54000 [==============================] - 22s - loss: 0.0122 - acc: 0.9978    
Epoch 12/20
54000/54000 [==============================] - 22s - loss: 0.0121 - acc: 0.9978    
Epoch 13/20
54000/54000 [==============================] - 22s - loss: 0.0122 - acc: 0.9978    
Epoch 14/20
54000/54000 [==============================] 

54000/54000 [==============================] - 24s - loss: 0.0124 - acc: 0.9978    
Epoch 8/20
54000/54000 [==============================] - 24s - loss: 0.0123 - acc: 0.9978    
Epoch 9/20
54000/54000 [==============================] - 24s - loss: 0.0122 - acc: 0.9979    
Epoch 10/20
54000/54000 [==============================] - 24s - loss: 0.0122 - acc: 0.9978    
Epoch 11/20
54000/54000 [==============================] - 24s - loss: 0.0122 - acc: 0.9978    
Epoch 12/20
54000/54000 [==============================] - 24s - loss: 0.0122 - acc: 0.9978    
Epoch 13/20
54000/54000 [==============================] - 24s - loss: 0.0122 - acc: 0.9978    
Epoch 14/20
54000/54000 [==============================] - 24s - loss: 0.0121 - acc: 0.9978    
Epoch 15/20
54000/54000 [==============================] - 24s - loss: 0.0121 - acc: 0.9978    
Epoch 16/20
54000/54000 [==============================] - 24s - loss: 0.0121 - acc: 0.9978    
Epoch 17/20
54000/54000 [=============================

...

54000/54000 [==============================] - 9s - loss: 0.0129 - acc: 0.9978     
Epoch 14/20
54000/54000 [==============================] - 9s - loss: 0.0129 - acc: 0.9978     
Epoch 15/20
54000/54000 [==============================] - 9s - loss: 0.0128 - acc: 0.9978     
Epoch 16/20
54000/54000 [==============================] - 9s - loss: 0.0128 - acc: 0.9978     
Epoch 17/20
54000/54000 [==============================] - 9s - loss: 0.0128 - acc: 0.9978     
Epoch 18/20
54000/54000 [==============================] - 9s - loss: 0.0128 - acc: 0.9978     
Epoch 19/20
54000/54000 [==============================] - 9s - loss: 0.0127 - acc: 0.9978     
Epoch 20/20
54000/54000 [==============================] - 14s - loss: 0.0779 - acc: 0.9979    
Epoch 2/20
54000/54000 [==============================] - 9s - loss: 0.0153 - acc: 0.9979     
Epoch 3/20
54000/54000 [==============================] - 9s - loss: 0.0142 - acc: 0.9979     
Epoch 4/20
54000/54000 [==============================

54000/54000 [==============================] - ETA: 0s - loss: 0.0127 - acc: 0.997 - 9s - loss: 0.0127 - acc: 0.9978     
Epoch 17/20
54000/54000 [==============================] - 9s - loss: 0.0127 - acc: 0.9978     
Epoch 18/20
54000/54000 [==============================] - 9s - loss: 0.0126 - acc: 0.9978     
Epoch 19/20
54000/54000 [==============================] - 9s - loss: 0.0126 - acc: 0.9978     
Epoch 20/20
54000/54000 [==============================] - 14s - loss: 0.0769 - acc: 0.9977    
Epoch 2/20
54000/54000 [==============================] - 9s - loss: 0.0158 - acc: 0.9977     
Epoch 3/20
54000/54000 [==============================] - 9s - loss: 0.0146 - acc: 0.9977     
Epoch 4/20
54000/54000 [==============================] - 9s - loss: 0.0141 - acc: 0.9977     
Epoch 5/20
54000/54000 [==============================] - 9s - loss: 0.0138 - acc: 0.9977     
Epoch 6/20
54000/54000 [==============================] - 9s - loss: 0.0136 - acc: 0.9977     
Epoch 7/20
54000/54

54000/54000 [==============================] - 8s - loss: 0.0142 - acc: 0.9978     
Epoch 20/20
54000/54000 [==============================] - 13s - loss: 0.0611 - acc: 0.9979    
Epoch 2/20
54000/54000 [==============================] - 8s - loss: 0.0159 - acc: 0.9979     
Epoch 3/20
54000/54000 [==============================] - 8s - loss: 0.0157 - acc: 0.9979     
Epoch 4/20
54000/54000 [==============================] - 8s - loss: 0.0153 - acc: 0.9979     
Epoch 5/20
54000/54000 [==============================] - 8s - loss: 0.0148 - acc: 0.9979     
Epoch 6/20
54000/54000 [==============================] - 8s - loss: 0.0146 - acc: 0.9979     
Epoch 7/20
54000/54000 [==============================] - 8s - loss: 0.0148 - acc: 0.9979     
Epoch 8/20
54000/54000 [==============================] - 8s - loss: 0.0148 - acc: 0.9979     
Epoch 9/20
54000/54000 [==============================] - 8s - loss: 0.0144 - acc: 0.9979     
Epoch 10/20
54000/54000 [==============================] - 8

54000/54000 [==============================] - 13s - loss: 0.0644 - acc: 0.9972    
Epoch 2/20
54000/54000 [==============================] - 8s - loss: 0.0163 - acc: 0.9978     
Epoch 3/20
54000/54000 [==============================] - 8s - loss: 0.0161 - acc: 0.9978     
Epoch 4/20
54000/54000 [==============================] - 8s - loss: 0.0156 - acc: 0.9978     
Epoch 5/20
54000/54000 [==============================] - 8s - loss: 0.0153 - acc: 0.9978     
Epoch 6/20
54000/54000 [==============================] - 8s - loss: 0.0152 - acc: 0.9978     
Epoch 7/20
54000/54000 [==============================] - 8s - loss: 0.0151 - acc: 0.9978     
Epoch 8/20
54000/54000 [==============================] - 8s - loss: 0.0146 - acc: 0.9978     
Epoch 9/20
54000/54000 [==============================] - 9s - loss: 0.0148 - acc: 0.9978     
Epoch 10/20
54000/54000 [==============================] - 8s - loss: 0.0146 - acc: 0.9978     
Epoch 11/20
54000/54000 [==============================] - 8

54000/54000 [==============================] - 9s - loss: 0.0151 - acc: 0.9979     
Epoch 5/20
54000/54000 [==============================] - 9s - loss: 0.0150 - acc: 0.9979     
Epoch 6/20
54000/54000 [==============================] - 9s - loss: 0.0153 - acc: 0.9979     
Epoch 7/20
54000/54000 [==============================] - 9s - loss: 0.0153 - acc: 0.9979     
Epoch 8/20
54000/54000 [==============================] - 9s - loss: 0.0144 - acc: 0.9979     
Epoch 9/20
54000/54000 [==============================] - 9s - loss: 0.0146 - acc: 0.9979     
Epoch 10/20
54000/54000 [==============================] - 9s - loss: 0.0142 - acc: 0.9979     
Epoch 11/20
54000/54000 [==============================] - 9s - loss: 0.0146 - acc: 0.9979     
Epoch 12/20
54000/54000 [==============================] - 9s - loss: 0.0144 - acc: 0.9979     
Epoch 13/20
54000/54000 [==============================] - 9s - loss: 0.0144 - acc: 0.9979     
Epoch 14/20
54000/54000 [==============================] 

In [21]:
best_accuracy

0.99780000000000002

In [22]:
best_parameters

{'activator': 'relu',
 'batch_size': 10,
 'epochs': 10,
 'initializer': 'uniform',
 'optimizer': 'adam'}

# OPTIMIZED MODEL 

In [23]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
epoch = 10 
classifier_opt = classifier_model(activator, optimizer, initializer)
classifier_opt.fit(X_train, y_train, batch_size = 10, epochs = epoch, verbose = 0)

In [24]:
# Predicting on the test set 
y_pred = classifier_opt.predict(X_test)
y_pred 

array([[ 0.00011542],
       [ 0.00118663],
       [ 0.00024391],
       ..., 
       [ 0.00011984],
       [ 0.00080167],
       [ 0.00021247]], dtype=float32)

In [25]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred.round(), average='weighted')

/Users/jpinzon/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.99652134716270879

In [26]:
y_pred.round()

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [28]:
y_classes = y_pred.argmax(axis=-1)
y_a = pd.DataFrame([y_classes])
y_b = pd.DataFrame([y_test])
y_c = pd.DataFrame([X_test_names])
y_a = pd.concat((y_a,y_b, y_c)).T
y_a.columns = ['predicted', 'actual','click_id']
y_a.head()

,predicted,actual,click_id
0,0,0,43660
1,0,0,87278
2,0,0,14317
3,0,0,81932
4,0,0,95321


In [30]:
# Accuracy on the test set
loss, accuracy = classifier_opt.evaluate(X_test, y_test, batch_size=10, verbose=0)
print("Accuracy = {:.4f}%, Loss = {:.4f}".format(accuracy* 100, loss))

Accuracy = 99.7680%, Loss = 0.0123


In [31]:
# Number of mismatches within the test set
y_a['dif'] = np.where(y_a.predicted == y_a.actual, 0, 1)
y_a[y_a['dif'] == 1].actual.value_counts().sort_index()

1    58
Name: actual, dtype: int64

In [34]:
classifier_opt.save('classifier.hdf5')
joblib.dump(sc, 'scaler.pkl') 

['scaler.pkl']